# Finetuning CASA

In [1]:
import os, sys
sys.path.append('../')
os.chdir('../')

import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertTokenizer, BertConfig, BertForPreTraining
from nltk.tokenize import TweetTokenizer, word_tokenize

from modules.multi_label_classification import BertForMultiLabelClassification
from utils.forward_fn import forward_sequence_multi_classification
from utils.metrics import absa_metrics_fn
from utils.data_utils import AspectBasedSentimentAnalysisProsaDataset, AspectBasedSentimentAnalysisDataLoader

In [2]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [3]:
# Set random seed
set_seed(26092020)

# Load Model

In [4]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = max(AspectBasedSentimentAnalysisProsaDataset.NUM_LABELS)
config.num_labels_list = AspectBasedSentimentAnalysisProsaDataset.NUM_LABELS

# Instantiate model
model = BertForMultiLabelClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Some weights of BertForMultiLabelClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifiers.3.bias', 'classifiers.5.weight', 'classifiers.2.bias', 'classifiers.1.weight', 'classifiers.0.weight', 'classifiers.5.bias', 'classifiers.1.bias', 'classifiers.0.bias', 'classifiers.4.bias', 'classifiers.3.weight', 'classifiers.2.weight', 'classifiers.4.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model

BertForMultiLabelClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [6]:
count_param(model)

124455186

# Prepare Dataset

In [7]:
train_dataset_path = './dataset/casa_absa-prosa/train_preprocess.csv'
valid_dataset_path = './dataset/casa_absa-prosa/valid_preprocess.csv'
test_dataset_path = './dataset/casa_absa-prosa/test_preprocess_masked_label.csv'

In [8]:
train_dataset = AspectBasedSentimentAnalysisProsaDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = AspectBasedSentimentAnalysisProsaDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = AspectBasedSentimentAnalysisProsaDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = AspectBasedSentimentAnalysisDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=True)  
valid_loader = AspectBasedSentimentAnalysisDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)  
test_loader = AspectBasedSentimentAnalysisDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)

In [9]:
w2i, i2w = AspectBasedSentimentAnalysisProsaDataset.LABEL2INDEX, AspectBasedSentimentAnalysisProsaDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'negative': 0, 'neutral': 1, 'positive': 2}
{0: 'negative', 1: 'neutral', 2: 'positive'}


# Test model on sample sentences

In [10]:
text = 'mesin 3SZ - VE 1500 cc ini memang lebih pas di badan Avanza, bertenaga namun hemat bahan bakar'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

print(f'Text: {text}')
for i, label in enumerate(labels):
    print(f'Label `{AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]}` : {i2w[label]} ({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)')

Text: mesin 3SZ - VE 1500 cc ini memang lebih pas di badan Avanza, bertenaga namun hemat bahan bakar
Label `fuel` : neutral (58.576%)
Label `machine` : negative (52.746%)
Label `others` : neutral (44.243%)
Label `part` : neutral (44.793%)
Label `price` : negative (48.917%)
Label `service` : positive (53.393%)


In [11]:
text = 'Jazz enak. tetapi mahal harga dan perawatan gila .'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

print(f'Text: {text}')
for i, label in enumerate(labels):
    print(f'Label `{AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]}` : {i2w[label]} ({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Jazz enak. tetapi mahal harga dan perawatan gila .
Label `fuel` : neutral (52.983%)
Label `machine` : negative (40.010%)
Label `others` : neutral (38.318%)
Label `part` : neutral (38.859%)
Label `price` : positive (40.025%)
Label `service` : positive (54.073%)


In [12]:
text = 'Toyota sekarang harga kemahalan, kualitas menurun, servicenya parah, dan boros bensin. '
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

print(f'Text: {text}')
for i, label in enumerate(labels):
    print(f'Label `{AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]}` : {i2w[label]} ({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Toyota sekarang harga kemahalan, kualitas menurun, servicenya parah, dan boros bensin. 
Label `fuel` : neutral (53.360%)
Label `machine` : negative (43.793%)
Label `others` : neutral (39.650%)
Label `part` : neutral (41.262%)
Label `price` : negative (46.365%)
Label `service` : positive (51.106%)


# Fine Tuning & Evaluation

In [13]:
optimizer = optim.Adam(model.parameters(), lr=5e-6)
model = model.cuda()

In [14]:
# Train
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_multi_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = absa_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_sequence_multi_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = absa_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = absa_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

  0%|          | 0/6 [00:00<?, ?it/s]

(Epoch 1) TRAIN LOSS:4.4927 ACC:0.74 F1:0.31 REC:0.33 PRE:0.32 LR:0.00000500


/home/samuel/anaconda2/envs/env_py3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
VALID LOSS:3.7081 ACC:0.79 F1:0.29 REC:0.33 PRE:0.26: 100%|██████████| 6/6 [00:00<00:00,  6.17it/s]
  0%|          | 0/51 [00:00<?, ?it/s]

(Epoch 1) VALID LOSS:3.7081 ACC:0.79 F1:0.29 REC:0.33 PRE:0.26


  0%|          | 0/6 [00:00<?, ?it/s]

...

(Epoch 9) VALID LOSS:1.4787 ACC:0.93 F1:0.82 REC:0.78 PRE:0.89


  0%|          | 0/6 [00:00<?, ?it/s]

(Epoch 10) TRAIN LOSS:0.8974 ACC:0.98 F1:0.95 REC:0.93 PRE:0.96 LR:0.00000500


VALID LOSS:1.4306 ACC:0.93 F1:0.84 REC:0.80 PRE:0.89: 100%|██████████| 6/6 [00:00<00:00,  6.16it/s]

(Epoch 10) VALID LOSS:1.4306 ACC:0.93 F1:0.84 REC:0.80 PRE:0.89


In [15]:
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_sequence_multi_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('pred.txt', index=False)

print(df)

100%|██████████| 12/12 [00:01<00:00, 11.60it/s]

     index                                              label
0        0  [neutral, neutral, positive, positive, neutral...
1        1  [neutral, neutral, positive, neutral, neutral,...
2        2  [neutral, neutral, neutral, positive, neutral,...
3        3  [neutral, neutral, positive, neutral, neutral,...
4        4  [neutral, neutral, positive, neutral, neutral,...
..     ...                                                ...
175    175  [neutral, neutral, neutral, neutral, neutral, ...
176    176  [neutral, neutral, neutral, negative, neutral,...
177    177  [neutral, neutral, neutral, negative, neutral,...
178    178  [neutral, neutral, neutral, negative, neutral,...
179    179  [positive, neutral, neutral, neutral, neutral,...

[180 rows x 2 columns]


# Test fine-tuned model on sample sentences

In [16]:
text = 'mesin 3SZ - VE 1500 cc ini memang lebih pas di badan Avanza, bertenaga namun hemat bahan bakar'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

print(f'Text: {text}')
for i, label in enumerate(labels):
    print(f'Label `{AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]}` : {i2w[label]} ({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)')

Text: mesin 3SZ - VE 1500 cc ini memang lebih pas di badan Avanza, bertenaga namun hemat bahan bakar
Label `fuel` : positive (50.618%)
Label `machine` : positive (91.462%)
Label `others` : neutral (93.839%)
Label `part` : neutral (88.449%)
Label `price` : neutral (95.410%)
Label `service` : neutral (97.029%)


In [17]:
text = 'Jazz enak. tetapi mahal harga dan perawatan gila .'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

print(f'Text: {text}')
for i, label in enumerate(labels):
    print(f'Label `{AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]}` : {i2w[label]} ({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Jazz enak. tetapi mahal harga dan perawatan gila .
Label `fuel` : neutral (97.261%)
Label `machine` : neutral (98.254%)
Label `others` : positive (52.839%)
Label `part` : neutral (96.773%)
Label `price` : negative (43.261%)
Label `service` : neutral (49.097%)


In [18]:
text = 'Toyota sekarang harga kemahalan, kualitas menurun, servicenya parah'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
labels = [torch.topk(logit, k=1, dim=-1)[1].squeeze().item() for logit in logits]

print(f'Text: {text}')
for i, label in enumerate(labels):
    print(f'Label `{AspectBasedSentimentAnalysisProsaDataset.ASPECT_DOMAIN[i]}` : {i2w[label]} ({F.softmax(logits[i], dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Toyota sekarang harga kemahalan, kualitas menurun, servicenya parah, dan boros bensin. 
Label `fuel` : neutral (78.270%)
Label `machine` : neutral (97.484%)
Label `others` : neutral (87.683%)
Label `part` : neutral (95.685%)
Label `price` : neutral (76.391%)
Label `service` : negative (71.735%)
